In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time, os, warnings
import seaborn as sns
from datetime import datetime, date, timedelta
from tqdm import trange
from sklearn.inspection import permutation_importance, partial_dependence
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import multilabel_confusion_matrix, classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
warnings.filterwarnings('ignore')

In [ ]:
final_data = pd.read_csv('Summarized_Data_UpToDate.csv', index_col = 0)
features = final_data.drop('behavioral_state', axis = 1)

In [ ]:
print(f"The Other state accounts for {final_data[final_data['behavioral_state'] == 0].shape[0] / len(final_data):.4f}")
print(f"The cover state accounts for {final_data[final_data['behavioral_state'] == 1].shape[0] / len(final_data):.4f}")
print(f"The hold on state accounts for {final_data[final_data['behavioral_state'] == 2].shape[0] / len(final_data):.4f}")
print(f"The evacuate state accounts for {final_data[final_data['behavioral_state'] == 3].shape[0] / len(final_data):.4f}")

In [ ]:
def evaluating(estimator, y_true, x_test):
    
    y_pred = estimator.predict(x_test)
    
    print(classification_report(y_true, y_pred))
    print(f"Accuracy is {accuracy_score(y_true, y_pred):.4f}")
    
    report = classification_report(y_true, y_pred, output_dict=True)
    return report

In [ ]:
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif.set_index('variables', inplace = True)
    return(vif)

In [ ]:
vif = calc_vif(features.drop(['DM_group_size','child_present'], axis = 1))

In [ ]:
vif

In [ ]:
features = features.drop(['DM_group_size','child_present'], axis = 1)

### Split train-test

In [ ]:
x = features.values
y = final_data['behavioral_state'].values
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, stratify=y, random_state=42)

### Hyperparameter tuning

#### MNL train set

In [ ]:
y, X = y_train, X_train
y = y.reshape(-1, 1) 
clf = LogisticRegression(random_state=200, multi_class = 'multinomial').fit(X, y)
print(clf.score(X, y))
ConfusionMatrixDisplay.from_estimator(clf, X, y)
# plt.savefig('../figures/multinomial_train.jpg', dpi = 150, bbox_inches = 'tight')

In [ ]:
report = evaluating(clf, y, X)
pd.DataFrame(report).transpose()

#### MNL test set

In [ ]:
y, X = y_test, X_test
y = y.reshape(-1, 1) 
clf = LogisticRegression(random_state=200, multi_class = 'multinomial').fit(X, y)
print(clf.score(X, y))
ConfusionMatrixDisplay.from_estimator(clf, X, y)
# plt.savefig('../figures/multinomial_test.jpg', dpi = 150, bbox_inches = 'tight')

In [ ]:
report = evaluating(clf, y, X)
pd.DataFrame(report).transpose()

#### XGBoost train set

In [ ]:
param_grid = {
    'n_estimators': np.arange(100, 701, 100),
    'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05],
    # Add other parameters here
    'max_depth': [4,5,6],
}

xgb = XGBClassifier(random_state = 200, importance_type = 'gain')
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs = -1)

grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Best Parameters: {best_parameters}")
print(f"Best Score: {best_score}")

best_model = grid_search.best_estimator_

In [ ]:
y, X = y_train, X_train
y = y.reshape(-1, 1) 
clf = XGBClassifier(learning_rate = 0.04, max_depth = 5, n_estimators = 100,
                    random_state=200, importance_type = 'gain', n_jobs = -1).fit(X, y)
print(clf.score(X, y))
ConfusionMatrixDisplay.from_estimator(clf, X, y)
# plt.savefig('../figures/xgboost_train.jpg', dpi = 150, bbox_inches = 'tight')

In [ ]:
report = evaluating(clf, y, X)
pd.DataFrame(report).transpose()

#### XGBoost test set

In [ ]:
y, X = y_test, X_test
y = y.reshape(-1, 1) 
clf = XGBClassifier(learning_rate = 0.04, max_depth = 5, n_estimators = 100,
                    random_state=200, importance_type = 'gain', n_jobs = -1).fit(X, y)
print(clf.score(X, y))

In [ ]:
y, X = y_test, X_test
y = y.reshape(-1, 1) 
clf = XGBClassifier(learning_rate = 0.04, max_depth = 5, n_estimators = 100,
                    random_state=200, importance_type = 'gain', n_jobs = -1).fit(X, y)
print(clf.score(X, y))
ConfusionMatrixDisplay.from_estimator(clf, X, y)
# plt.savefig('../figures/xgboost_test.jpg', dpi = 150, bbox_inches = 'tight')

In [ ]:
report = evaluating(clf, y, X)
pd.DataFrame(report).transpose()

### Feature importance

In [ ]:
# ['alarm_on', 'start_pos_DM', 'envir_crowded', 'shaking_intensity',
#        'obstacle_floor', 'DM_far_egress', 'cover_availability', 'time_elapsed',
#        'public_setting', 'num_people', 'DM_leader']

feature_names = ['Alarm_On', 'Start_Pos_DM', 'Envir_Crowded', 'Shaking_Intensity',
       'Obstacle_Floor', 'DM_Far_Egress', 'Cover_Availability', 'Time_Elapsed',
       'Public_Setting', 'Num_People', 'DM_Leader']

feature_names = ['Whether the alarm is on',
                 'Starting position of the decision-maker',
                 'Whether the environment is crowded',
                 'Shaking intensity',
                   'Whether there is obstacle on the floor',
                 'Whether the decision-maker is far from the egress',
                 'Whether there is cover in the environment',
                 'Time elapsed after shaking',
                'Whether it is a public setting',
                 'Number of people shown in the environment',
                 'Whether the decision-maker is a leader']

In [ ]:
#feature importances with "Gain" method
plt.close('all')
importances_RF = clf.feature_importances_*100 # train set
indices_RF = np.argsort(importances_RF)[::-1]

print("Feature ranking:")

for f in range(features.shape[1]):
     print("%d. col.%d %s (%f)" % (f + 1, indices_RF[f],
                                   features.columns[indices_RF[f]], importances_RF[indices_RF[f]]))

num_bars = 11
plt.figure(figsize=[8,8],dpi=200)
plt.title("")
plt.barh(range(num_bars), importances_RF[indices_RF[range(num_bars-1,-1,-1)]])
plt.yticks(range(num_bars), feature_names)
plt.ylim([-1, num_bars])
plt.xlim([0,20])
plt.xlabel("Relative Feature importance (%)")
# plt.ylabel("Feature")
plt.grid(b=1,linestyle='--')
for a, b in enumerate(importances_RF[indices_RF[range(num_bars-1,-1,-1)]]):
    b=round(b,2)
    plt.text(b+0.75, a-0.3, '%s' % format(b,'.2f'), ha='center', va='bottom')
plt.savefig('../../USGS/figures/xgb_feature_importance.jpg', dpi = 200, bbox_inches = 'tight')
plt.show()

## PDP

In [ ]:
class_names = ['Other','Drop and Cover', 'Hold on', 'Evacuate']

In [ ]:
def value_grid(data, feature, grid_size):
    max_value = data[feature].max()
    min_value = data[feature].min()
    values = np.linspace(min_value, max_value, grid_size)
    return values

def value_unique(data, feature):
    values = data[feature].sort_values().unique()
    return values

def make_predictions(data_x, feature, value_grid, estimator):
    every_value_prediction = []
    for i in range(len(value_grid)):
        this_prediction_data = data_x.copy()
        this_prediction_data[feature] = value_grid[i]
        this_prediction = estimator.predict_proba(this_prediction_data)
        every_value_prediction.append(this_prediction)
    return every_value_prediction

def get_average(every_value_prediction):
    predictions = []
    for i in range(len(every_value_prediction)):
        predictions.append([])
        for j in range(n_class):
            predictions[i].append(every_value_prediction[i][:, j].mean())
    return np.array(predictions)

In [ ]:
def get_partial_dependence_unique():

    partial_dependece = {}

    values = value_unique(data, feature)

    every_value_prediction = make_predictions(data, feature, values, estimator)

    average_prediction = get_average(every_value_prediction)

    partial_dependece['average'] = np.array(average_prediction)

    partial_dependece['values'] = values

    return partial_dependece

def get_partial_dependence_continuous():

    partial_dependece = {}

    values = value_grid(data, feature, grid_size)

    every_value_prediction = make_predictions(data, feature, values, estimator)

    average_prediction = get_average(every_value_prediction)

    partial_dependece['average'] = np.array(average_prediction)

    partial_dependece['values'] = values

    return partial_dependece

In [ ]:
def plot_pdp(pdp_dic, name, xlabel):
    for j in range(n_class):
        plt.plot(pdp_dic[name]['values'], pdp_dic[name]['average'][:, j],color='k')
        sns.rugplot(features[name], color='k')
        plt.ylabel('Probability for {}'.format(class_names[j]),fontsize=15)
        plt.xlabel(xlabel,fontsize=15)
        plt.xticks(fontsize=13)
        plt.yticks(fontsize=13)
        plt.savefig('../../USGS/figures/pdp/xgboost/{} for class {}.jpg'.format(name, class_names[j]),
                    dpi=150,bbox_inches='tight')
        plt.show()

In [ ]:
features.columns

In [ ]:
continuous_features = ['time_elapsed']
unique_features = ['shaking_intensity', 'envir_crowded',
       'cover_availability', 'obstacle_floor', 'start_pos_DM', 'num_people',
                   'public_setting', 'alarm_on', 'DM_leader',
       'DM_far_egress']
class_names = ['Other','Drop and cover', 'Hold on', 'Evacuate']
n_class = 4
grid_size = 50
estimator = clf

In [ ]:
data = features

start = time.time()

continuous_partial_dependence_dic = {}

for feature in continuous_features:
    
    print('{} starts'.format(feature))
    print('  ')

    continuous_partial_dependence_dic[feature] = get_partial_dependence_continuous()

    print('time elapsed: ')
    print(time.time() - start)
    print('{} ends'.format(feature))
    
continuous_partial_dependence_dic

In [ ]:
plot_pdp(continuous_partial_dependence_dic, 'time_elapsed', 'Time elapsed after shaking')

In [ ]:
data = features

start = time.time()

unique_partial_dependence_dic = {}

for feature in unique_features:
    
    print('{} starts'.format(feature))
    print('  ')

    unique_partial_dependence_dic[feature] = get_partial_dependence_unique()

    print('time elapsed: ')
    print(time.time() - start)
    print('{} ends'.format(feature))
     
unique_partial_dependence_dic

In [ ]:
plot_pdp(unique_partial_dependence_dic, 'shaking_intensity', 'Shaking intensity')

In [ ]:
plot_pdp(unique_partial_dependence_dic, 'start_pos_DM', 'Starting position of the decision-maker')

In [ ]:
plot_pdp(unique_partial_dependence_dic, 'num_people', 'Number of people shown in the environment')

In [ ]:
def plot_dummy(pdp_dic, name, xlabel):
    for j in range(n_class):
        plt.scatter(pdp_dic[name]['values'], pdp_dic[name]['average'][:, j],color='k')
        sns.rugplot(features[name], color='k')
        plt.ylabel('Probability for {}'.format(class_names[j]),fontsize=15)
        plt.xlabel(xlabel,fontsize=15)
        plt.xticks(fontsize=13)
        plt.yticks(fontsize=13)
        plt.savefig('../../USGS/figures/pdp/xgboost/{} for class {}.jpg'.format(name, class_names[j]),
                    dpi=150,bbox_inches='tight')
        plt.show()

In [ ]:
plot_dummy(unique_partial_dependence_dic, 'DM_leader', 'Whether the decision-maker is a leader')

In [ ]:
plot_dummy(unique_partial_dependence_dic, 'public_setting', 'Whether it is a public setting')

In [ ]:
plot_dummy(unique_partial_dependence_dic, 'envir_crowded', 'Whether the environment is crowded')

In [ ]:
plot_dummy(unique_partial_dependence_dic, 'obstacle_floor', 'Whether there is obstacle on the floor')

In [ ]:
plot_dummy(unique_partial_dependence_dic, 'DM_far_egress', 'Whether the decision-maker is far from the egress')

In [ ]:
plot_dummy(unique_partial_dependence_dic, 'cover_availability', 'Whether there is cover in the environment')

In [ ]:
plot_dummy(unique_partial_dependence_dic, 'alarm_on', 'Whether the alarm is on')                                                                                                                                                                                                                                                                                                                                                                                              